In [367]:
import requests
import json
import pandas as pd
from getpass import getpass
from pymongo import MongoClient
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import pandas as pd
from math import sin, cos, sqrt, atan2, radians

In [368]:
import os
from dotenv import load_dotenv
load_dotenv()

token = os.getenv("token")

In [369]:
client = MongoClient("localhost:27017")
client.list_database_names()
db = client["Ironhack"].get_collection("companies")

In [370]:
# Gamer's Galaxy Studios
    
    # San Francisco (RockYou)
sanfrancisco = [37.790167, -122.417359, 37.619213, -122.382645]

    # Honolulu
honolulu = [21.306944, -157.860274, 21.318499, -157.925735]

    # Chicago
chicago = [41.884518, -87.632395, 41.784455, -87.752370]

In [371]:
# ¡¡ ¿ WHERE ? !!

location = chicago

In [372]:
### 1. Gaming companies nearby.


# From Mongo, return a dictionary with all the companies that meet the criteria: It is a gaming company and is in the US.

gamings = list(db.find({"offices.country_code":"USA", "category_code":"games_video", "offices.latitude":{"$exists":"True"}, "offices.longitude":{"$exists":"True"}}, projection = {"_id":0, "name":1, "offices.country_code":1, "offices.address1":1, "offices.longitude":1, "offices.latitude":1}))


# Create different lists to get the names and coordinates of all gaming companies in the US.

gamingname = []
gaminglat = []
gaminglon = []

for i in range(len(gamings)):
    if (gamings[i]["offices"][0]["latitude"]) is not None and (gamings[i]["offices"][0]["longitude"] is not None):
        gamingname.append(gamings[i]["name"])
        gaminglat.append(gamings[i]["offices"][0]["latitude"])
        gaminglon.append(gamings[i]["offices"][0]["longitude"])


# Create a dictionary that will store all this data for later on convert to a DataFrame.

gamingcompanies = {}

gamingcompanies["name"] = gamingname
gamingcompanies["lat"] = gaminglat
gamingcompanies["lon"] = gaminglon


# We are going to now work on finding the distance between our given location (where we want to locate our new office) and
# all the gaming companies in the US. Order them by distance to get the closest ones.

gamingdistance = []

for i, j in zip(gaminglat, gaminglon):
    
    r = 6373.0

    lat1 = radians(location[0])
    lon1 = radians(location[1])
    lat2 = radians(i)
    lon2 = radians(j)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    x = r * c * 1000
    
    gamingdistance.append(x)

gamingcompanies["dist"] = gamingdistance


# Convert the dictionary with all the information into a DataFrame and print the 5 closest ones.

dfgaming = pd.DataFrame(gamingcompanies)
dfgaming = dfgaming.sort_values(by=['dist'])
dfgaming = dfgaming[dfgaming["dist"] < 1000]
dfgaming = dfgaming.drop_duplicates(subset=['name'])
gamingdataframe = dfgaming

gamingdataframe

,name,lat,lon,dist
317,SureSpeak,41.889183,-87.632519,518.988757
436,Midway Games,41.879535,-87.624333,867.722704


In [373]:
### 2. Schools nearby.


# Function to get schools from FSQ that are near a given location.

def requests_for_foursquare (query, lat, lon, radius=1000, limit=30):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

schools = requests_for_foursquare("schools", location[0], location[1])

In [374]:
# Create the corresponding lists to then make them a dataframe where we store the necessary data.

schoolname = []
schooldistance = []
schoollatitude = []
schoollongitude = []
schooltype = []

for i in range(len(schools["results"])):
    schooltype.append(schools["results"][i]["categories"][0]["name"])
    schoolname.append(schools["results"][i]["name"])
    schooldistance.append(schools["results"][i]["distance"])
    schoollatitude.append(schools["results"][i]["geocodes"]["main"]["latitude"])
    schoollongitude.append(schools["results"][i]["geocodes"]["main"]["longitude"])

school = {}

school["name"] = schoolname
school["latitude"] = schoollatitude
school["longitude"] = schoollongitude
school["type"] = schooltype
school["distance"] = schooldistance

schoolsdataframe = pd.DataFrame(school)

schoolsdataframe

,name,latitude,longitude,type,distance
0,Law Offices of Michael Baker,41.882839,-87.632112,Law Office,190
1,Digital Professional Institute,41.882754,-87.631179,Primary and Secondary School,220
2,Learncore,41.886477,-87.633158,Education,226
3,Stepping Stones Together,41.882316,-87.631122,General Contractor,266
4,DeVry University,41.882868,-87.634926,Community College,278
5,iRepair Chicago,41.882856,-87.635012,College Technology Building,282
6,DeVry University Chicago Loop Center,41.882902,-87.635073,College and University,288
7,Bright Horizons at Cook County/City of Chicago...,41.882690,-87.629507,Daycare,295
8,Landmark Education,41.886660,-87.634938,Education,317
9,Northern Il University,41.881818,-87.631090,College and University,318


In [375]:
### 3. Nearby start-ups with a +$1M raising fund.

# What is a start-up?
    # < 10 years since founded
    # < 100 employees

startups = list(db.find({"offices.country_code":"USA", "name":"Fixya", "number_of_employees":{"$lte":100}, "founded_year":{"$exists":"True"}, "founded_year":{"$gte":2013}, "offices.latitude":{"$exists":"True"}, "offices.longitude":{"$exists":"True"}}, projection = {"_id":0, "name":1, "offices.country_code":1, "offices.address1":1, "offices.longitude":1, "offices.latitude":1, "total_money_raised":1}))

startupname = [startups[0]["name"]]
startuplatitude = [startups[0]["offices"][0]["latitude"]]
startuplongitude = [startups[0]["offices"][0]["longitude"]]

startup = {}

startup["name"] = startupname
startup["lat"] = startuplatitude
startup["lon"] = startuplongitude

startupdistance = []

for i, j in zip(startuplatitude, startuplongitude):
    
    r = 6373.0

    lat1 = radians(location[0])
    lon1 = radians(location[1])
    lat2 = radians(i)
    lon2 = radians(j)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    x = r * c * 1000
    
    startupdistance.append(x)

startup["dist"] = startupdistance

dfstartup = pd.DataFrame(startup)
dfstartup = dfstartup[dfstartup["dist"] < 1000]
startupdataframe = dfstartup

startupdataframe

,name,lat,lon,dist


In [376]:
### 4. Starbucks nearby.


# Function to get schools from FSQ that are near a given location.

def requests_for_foursquare (query, lat, lon, radius=1000, limit=30):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

starbucks = requests_for_foursquare("starbucks", location[0], location[1])

In [377]:
# Create the corresponding lists to then make them a dataframe where we store the necessary data.

starbucksname = []
starbucksdistance = []
starbuckslatitude = []
starbuckslongitude = []

for i in range(len(starbucks["results"])):
    starbucksname.append(starbucks["results"][i]["name"])
    starbucksdistance.append(starbucks["results"][i]["distance"])
    starbuckslatitude.append(starbucks["results"][i]["geocodes"]["main"]["latitude"])
    starbuckslongitude.append(starbucks["results"][i]["geocodes"]["main"]["longitude"])

starbuck = {}

starbuck["name"] = starbucksname
starbuck["latitude"] = starbuckslatitude
starbuck["longitude"] = starbuckslongitude
starbuck["distance"] = starbucksdistance

starbucksdataframe = pd.DataFrame(starbuck)

starbucksdataframe

,name,latitude,longitude,distance
0,Starbucks,41.885355,-87.632825,122
1,Starbucks,41.884186,-87.633561,122
2,Starbucks,41.882890,-87.632898,189
3,Starbucks,41.882295,-87.634281,293
4,Starbucks,41.883812,-87.628551,306
5,Starbucks,41.885882,-87.628886,330
6,Starbucks,41.880824,-87.632157,414
7,Starbucks,41.881589,-87.630379,431
8,Starbucks,41.888599,-87.635410,492
9,Starbucks,41.880322,-87.634792,497


In [378]:
# 5. Transportation

def requests_for_foursquare (query, lat, lon, radius=1000, limit=30):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

travels = requests_for_foursquare("station", location[0], location[1])

In [379]:
travelname = []
traveldistance = []
travellat = []
travellon = []

for i in range(len(travels["results"])):
    if ("Bus" or "Trail") in travels["results"][i]["categories"][0]["name"]:
        travelname.append(travels["results"][i]["categories"][0]["name"])
        traveldistance.append(travels["results"][i]["distance"])
        travellat.append(travels["results"][i]["geocodes"]["main"]["latitude"])
        travellon.append(travels["results"][i]["geocodes"]["main"]["longitude"])


# Airport

travelname.append("airport")
travellat.append(location[2])
travellon.append(location[3])


# Distance to airport.

r = 6373.0

lat1 = radians(location[0])
lon1 = radians(location[1])
lat2 = radians(location[2])
lon2 = radians(location[3])

dlon = lon2 - lon1
dlat = lat2 - lat1

a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
c = 2 * atan2(sqrt(a), sqrt(1 - a))

x = r * c * 1000

traveldistance.append(x)

travelling = {}

travelling["name"] = travelname
travelling["lat"] = travellat
travelling["lon"] = travellon
travelling["dist"] = traveldistance

travellingdataframe = pd.DataFrame(travelling)

travellingdataframe

,name,lat,lon,dist
0,Bus Station,41.882957,-87.629800,275.000000
1,Bus Station,41.881725,-87.632099,316.000000
2,Bus Station,41.880849,-87.629723,474.000000
3,Bus Station,41.888798,-87.633307,480.000000
4,airport,41.784455,-87.752370,14924.385981


In [380]:
# 6. Clubs to party nearby.

def requests_for_foursquare (query, lat, lon, radius=1000, limit=30):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

clubs = requests_for_foursquare("club", location[0], location[1])

In [381]:
clubsname = []
clubstype = []
clubslatitude = []
clubslongitude = []
clubsdistance = []

for i in range(len(clubs["results"])):
    if (clubs["results"][i]["categories"][0]["name"] != ("Social Club") and clubs["results"][i]["categories"][0]["name"] != ("Sports Club") and clubs["results"][i]["categories"][0]["name"] != ("Comedy Club")) and ("Bar" in clubs["results"][i]["categories"][0]["name"] or "Club" in clubs["results"][i]["categories"][0]["name"]):
        clubsname.append(clubs["results"][i]["name"])
        clubstype.append(clubs["results"][i]["categories"][0]["name"])
        clubsdistance.append(clubs["results"][i]["distance"])
        clubslatitude.append(clubs["results"][i]["geocodes"]["main"]["latitude"])
        clubslongitude.append(clubs["results"][i]["geocodes"]["main"]["longitude"])

clubdict = {}

clubdict["name"] = clubsname
clubdict["lat"] = clubslatitude
clubdict["lon"] = clubslongitude
clubdict["distance"] = clubsdistance
clubdict["type"] = clubstype

clubsdataframe = pd.DataFrame(clubdict)
clubsdataframe

,name,lat,lon,distance,type
0,Pure Platinum,41.882805,-87.636488,388,Night Club
1,Tunnel,41.889062,-87.633034,520,Night Club
2,Dance Studio,41.889423,-87.630215,561,Night Club
3,Disco,41.889776,-87.631601,610,Night Club
4,Ladies Of Chicago,41.880369,-87.626938,645,Strip Club
5,Chicago's First Lady,41.880598,-87.626518,652,Strip Club
6,Chicago Dancers United,41.878435,-87.633975,688,Night Club
7,LondonHouse Chicago,41.887803,-87.625327,689,Night Club
8,The Underground,41.890941,-87.630139,729,Night Club
9,Andy's Jazz Club,41.889891,-87.627433,730,Night Club


In [382]:
# 7. Vegan restaurants/supermarkets.

def requests_for_foursquare (query, lat, lon, radius=1000, limit=30):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

vegan = requests_for_foursquare("vegan", location[0], location[1])

In [383]:
veganname = []
vegantype = []
veganlatitude = []
veganlongitude = []
vegandistance = []

for i in range(len(vegan["results"])):
    if "Vegan" in vegan["results"][i]["name"]:
        veganname.append(vegan["results"][i]["name"])
        vegantype.append(vegan["results"][i]["categories"][0]["name"])
        vegandistance.append(vegan["results"][i]["distance"])
        veganlatitude.append(vegan["results"][i]["geocodes"]["main"]["latitude"])
        veganlongitude.append(vegan["results"][i]["geocodes"]["main"]["longitude"])

vegandict = {}

vegandict["name"] = veganname
vegandict["lat"] = veganlatitude
vegandict["lon"] = veganlongitude
vegandict["distance"] = vegandistance
vegandict["type"] = vegantype

vegandataframe = pd.DataFrame(vegandict)
vegandataframe

,name,lat,lon,distance,type


In [384]:
# 8. Basketball courts.

def requests_for_foursquare (query, lat, lon, radius=1000, limit=30):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

basketball = requests_for_foursquare("basket", location[0], location[1])

In [385]:
basketname = []
basketlatitude = []
basketlongitude = []
basketdistance = []

for i in range(len(basketball["results"])):
    if "Basketball" in basketball["results"][i]["name"]:
        basketname.append(basketball["results"][i]["name"])
        basketdistance.append(basketball["results"][i]["distance"])
        basketlatitude.append(basketball["results"][i]["geocodes"]["main"]["latitude"])
        basketlongitude.append(basketball["results"][i]["geocodes"]["main"]["longitude"])

basketdict = {}

basketdict["name"] = basketname
basketdict["lat"] = basketlatitude
basketdict["lon"] = basketlongitude
basketdict["distance"] = basketdistance

basketballdataframe = pd.DataFrame(basketdict)
basketballdataframe

,name,lat,lon,distance
0,Steam Basketball,41.879803,-87.626612,709
1,National Basketball Retired,41.877933,-87.633218,734
2,Legends of Basketball,41.877827,-87.633182,746
3,Grand Plaza Basketball Court,41.891953,-87.628580,878
4,Alta at K Station Basketball/Volleyball Court,41.888807,-87.642673,980


In [386]:
# 9. Dog Care.

def requests_for_foursquare (query, lat, lon, radius=1000, limit=10):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}&sort=DISTANCE"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

dogs = requests_for_foursquare("dog", location[0], location[1])

In [387]:
dogsname = []
dogslatitude = []
dogslongitude = []
dogsdistance = []

for i in range(len(dogs["results"])):
    if "Care" in dogs["results"][i]["name"]:
        dogsname.append(dogs["results"][i]["name"])
        dogsdistance.append(dogs["results"][i]["distance"])
        dogslatitude.append(dogs["results"][i]["geocodes"]["main"]["latitude"])
        dogslongitude.append(dogs["results"][i]["geocodes"]["main"]["longitude"])

dogsdict = {}

dogsdict["name"] = dogsname
dogsdict["lat"] = dogslatitude
dogsdict["lon"] = dogslongitude
dogsdict["distance"] = dogsdistance

dogcaredataframe = pd.DataFrame(dogsdict)
dogcaredataframe

,name,lat,lon,distance


In [581]:
# 0.1 Map and office location

map = Map(location = [location[0], location[1]], zoom_start=15)


# 0.2 Office Location

officeicon = Icon(
    color = "red",
    icon_color = "white",
    icon = "building",
    prefix = "fa" #font-awesome: fa
)

officemarker = Marker(location = [location[0], location[1]], tooltip="The Office", icon=officeicon)
officemarker.add_to(map)


# 1. gamingdataframe

for a, b in gamingdataframe.iterrows():

    gamingicon = Icon(
        color = "purple",
        icon_color = "white",
        icon = "gamepad",
        prefix = "fa" #font-awesome: fa
)

    gamingmarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=gamingicon)
    gamingmarker.add_to(map)


# 2. schoolsdataframe

for a, b in schoolsdataframe.iterrows():

    schoolsicon = Icon(
        color = "blue",
        icon_color = "white",
        icon = "graduation-cap",
        prefix = "fa" #font-awesome: fa
)

    schoolsmarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=schoolsicon)
    schoolsmarker.add_to(map)


# 3. startupdataframe

for a, b in startupdataframe.iterrows():

    startupicon = Icon(
        color = "black",
        icon_color = "white",
        icon = "file-text",
        prefix = "fa" #font-awesome: fa
)

    startupmarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=startupicon)
    startupmarker.add_to(map)


# 4. starbucksdataframe

for a, b in starbucksdataframe.iterrows():

    starbucksicon = Icon(
        color = "green",
        icon_color = "white",
        icon = "coffee",
        prefix = "fa" #font-awesome: fa
)

    starbucksmarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=starbucksicon)
    starbucksmarker.add_to(map)


# 5. travellingdataframe

for a, b in travellingdataframe.iterrows():

    travellingicon = Icon(
        color = "gray",
        icon_color = "white",
        icon = "suitcase",
        prefix = "fa" #font-awesome: fa
)

    travellingmarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=travellingicon)
    travellingmarker.add_to(map)


# 6. clubsdataframe

for a, b in clubsdataframe.iterrows():

    clubsicon = Icon(
        color = "cadetblue",
        icon_color = "white",
        icon = "beer",
        prefix = "fa" #font-awesome: fa
)

    clubsmarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=clubsicon)
    clubsmarker.add_to(map)


# 7. vegandataframe

for a, b in vegandataframe.iterrows():

    veganicon = Icon(
        color = "lightgreen",
        icon_color = "white",
        icon = "leaf",
        prefix = "fa" #font-awesome: fa
)

    veganmarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=veganicon)
    veganmarker.add_to(map)


# 8. basketballdataframe

for a, b in basketballdataframe.iterrows():

    basketballicon = Icon(
        color = "orange",
        icon_color = "white",
        icon = "trophy",
        prefix = "fa" #font-awesome: fa
)

    basketballmarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=basketballicon)
    basketballmarker.add_to(map)


# 9. dogcaredataframe

for a, b in dogcaredataframe.iterrows():

    dogcareicon = Icon(
        color = "darkgreen",
        icon_color = "white",
        icon = "scissors",
        prefix = "fa", #font-awesome: fa
)

    dogcaremarker = Marker(location = [b[1], b[2]], tooltip=b[0], icon=dogcareicon)
    dogcaremarker.add_to(map)

folium.CircleMarker(
    location=[location[0],location[1]],
    radius=270,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.15
).add_to(map)

map

In [456]:
sanfranciscogeo = {
  "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.41722395556843,
              37.799048121573904
            ],
            [
              -122.41834379850313,
              37.79900464581997
            ],
            [
              -122.41945285281575,
              37.798874637406115
            ],
            [
              -122.42054043393536,
              37.79865934883657
            ],
            [
              -122.4215960643854,
              37.79836085419215
            ],
            [
              -122.42260957482134,
              37.797982029127496
            ],
            [
              -122.42357120208383,
              37.797526523138956
            ],
            [
              -122.42447168331967,
              37.79699872437169
            ],
            [
              -122.42530234525925,
              37.796403717306525
            ],
            [
              -122.426055187789,
              37.79574723373641
            ],
            [
              -122.42672296101132,
              37.79503559750653
            ],
            [
              -122.42729923505046,
              37.79427566355265
            ],
            [
              -122.42777846193125,
              37.793474751827034
            ],
            [
              -122.42815602893793,
              37.792640576750635
            ],
            [
              -122.42842830294079,
              37.791781172872675
            ],
            [
              -122.42859266526797,
              37.79090481745567
            ],
            [
              -122.42864753679117,
              37.790019950732706
            ],
            [
              -122.42859239298859,
              37.78913509460604
            ],
            [
              -122.42842776884562,
              37.7882587705707
            ],
            [
              -122.42815525355194,
              37.78739941765381
            ],
            [
              -122.42777747505207,
              37.78656531115948
            ],
            [
              -122.42729807460326,
              37.78576448300136
            ],
            [
              -122.42672167159151,
              37.785004644388934
            ],
            [
              -122.42605381894818,
              37.784293111610566
            ],
            [
              -122.42530094960128,
              37.78363673562644
            ],
            [
              -122.42447031447885,
              37.78304183614724
            ],
            [
              -122.42356991266396,
              37.782514140831495
            ],
            [
              -122.42260841437408,
              37.78205873018443
            ],
            [
              -122.42159507750617,
              37.78167998868727
            ],
            [
              -122.4205396585493,
              37.781381562624915
            ],
            [
              -122.41945231872056,
              37.78116632501644
            ],
            [
              -122.41834352622378,
              37.78103634798426
            ],
            [
              -122.41722395556843,
              37.780992882826624
            ],
            [
              -122.41610438491308,
              37.78103634798426
            ],
            [
              -122.4149955924163,
              37.78116632501644
            ],
            [
              -122.41390825258755,
              37.781381562624915
            ],
            [
              -122.4128528336307,
              37.78167998868727
            ],
            [
              -122.41183949676275,
              37.78205873018443
            ],
            [
              -122.41087799847287,
              37.782514140831495
            ],
            [
              -122.409977596658,
              37.78304183614724
            ],
            [
              -122.40914696153558,
              37.78363673562644
            ],
            [
              -122.40839409218866,
              37.784293111610566
            ],
            [
              -122.40772623954535,
              37.785004644388934
            ],
            [
              -122.40714983653358,
              37.78576448300136
            ],
            [
              -122.4066704360848,
              37.78656531115948
            ],
            [
              -122.4062926575849,
              37.78739941765381
            ],
            [
              -122.40602014229123,
              37.7882587705707
            ],
            [
              -122.40585551814826,
              37.78913509460604
            ],
            [
              -122.40580037434566,
              37.790019950732706
            ],
            [
              -122.40585524586886,
              37.79090481745567
            ],
            [
              -122.40601960819605,
              37.791781172872675
            ],
            [
              -122.4062918821989,
              37.792640576750635
            ],
            [
              -122.4066694492056,
              37.793474751827034
            ],
            [
              -122.40714867608641,
              37.79427566355265
            ],
            [
              -122.40772495012551,
              37.79503559750653
            ],
            [
              -122.40839272334786,
              37.79574723373641
            ],
            [
              -122.4091455658776,
              37.796403717306525
            ],
            [
              -122.40997622781717,
              37.79699872437169
            ],
            [
              -122.410876709053,
              37.797526523138956
            ],
            [
              -122.41183833631553,
              37.797982029127496
            ],
            [
              -122.41285184675144,
              37.79836085419215
            ],
            [
              -122.41390747720149,
              37.79865934883657
            ],
            [
              -122.41499505832108,
              37.798874637406115
            ],
            [
              -122.4161041126337,
              37.79900464581997
            ],
            [
              -122.41722395556843,
              37.799048121573904
            ]
          ]
        ]
      }
    }

chicagogeo = {
"geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -87.6324397541506,
              41.893493809166905
            ],
            [
              -87.6336272313313,
              41.8934503776852
            ],
            [
              -87.6348032676522,
              41.89332050168519
            ],
            [
              -87.63595653262,
              41.893105432465575
            ],
            [
              -87.63707591540441,
              41.89280724210791
            ],
            [
              -87.6381506320053,
              41.89242880348867
            ],
            [
              -87.63917032925194,
              41.891973762568114
            ],
            [
              -87.64012518462746,
              41.89144650322421
            ],
            [
              -87.64100600095236,
              41.89085210497255
            ],
            [
              -87.64180429501228,
              41.89019629398175
            ],
            [
              -87.64251237927404,
              41.889485387858485
            ],
            [
              -87.64312343590231,
              41.888726234736815
            ],
            [
              -87.64363158236523,
              41.88792614726077
            ],
            [
              -87.64403192799863,
              41.88709283209871
            ],
            [
              -87.64432062098805,
              41.88623431567057
            ],
            [
              -87.64449488532023,
              41.885358866805234
            ],
            [
              -87.6445530473539,
              41.8844749170747
            ],
            [
              -87.64449455176026,
              41.883590979573505
            ],
            [
              -87.64431996668662,
              41.88271556692617
            ],
            [
              -87.64403097810016,
              41.881857109312904
            ],
            [
              -87.64363037337378,
              41.881023873302354
            ],
            [
              -87.64312201427876,
              41.880223882272674
            ],
            [
              -87.64251079965057,
              41.87946483918588
            ],
            [
              -87.64180261809294,
              41.878754052457424
            ],
            [
              -87.64100429118021,
              41.87809836563307
            ],
            [
              -87.64012350770808,
              41.87750409154787
            ],
            [
              -87.63916874962844,
              41.87697695159876
            ],
            [
              -87.6381492103817,
              41.8765220207131
            ],
            [
              -87.63707470641292,
              41.876143678540245
            ],
            [
              -87.63595558272146,
              41.875845567334075
            ],
            [
              -87.63480261335071,
              41.87563055692933
            ],
            [
              -87.63362689777132,
              41.87550071714734
            ],
            [
              -87.6324397541506,
              41.87545729789497
            ],
            [
              -87.6312526105299,
              41.87550071714734
            ],
            [
              -87.63007689495049,
              41.87563055692933
            ],
            [
              -87.62892392557974,
              41.875845567334075
            ],
            [
              -87.62780480188829,
              41.876143678540245
            ],
            [
              -87.62673029791951,
              41.8765220207131
            ],
            [
              -87.62571075867277,
              41.87697695159876
            ],
            [
              -87.62475600059312,
              41.87750409154787
            ],
            [
              -87.623875217121,
              41.87809836563307
            ],
            [
              -87.62307689020827,
              41.878754052457424
            ],
            [
              -87.62236870865065,
              41.87946483918588
            ],
            [
              -87.62175749402245,
              41.880223882272674
            ],
            [
              -87.62124913492742,
              41.881023873302354
            ],
            [
              -87.62084853020104,
              41.881857109312904
            ],
            [
              -87.62055954161458,
              41.88271556692617
            ],
            [
              -87.62038495654096,
              41.883590979573505
            ],
            [
              -87.62032646094733,
              41.8844749170747
            ],
            [
              -87.62038462298096,
              41.885358866805234
            ],
            [
              -87.62055888731315,
              41.88623431567057
            ],
            [
              -87.6208475803026,
              41.88709283209871
            ],
            [
              -87.62124792593599,
              41.88792614726077
            ],
            [
              -87.6217560723989,
              41.888726234736815
            ],
            [
              -87.62236712902718,
              41.889485387858485
            ],
            [
              -87.62307521328893,
              41.89019629398175
            ],
            [
              -87.62387350734885,
              41.89085210497255
            ],
            [
              -87.62475432367377,
              41.89144650322421
            ],
            [
              -87.62570917904927,
              41.891973762568114
            ],
            [
              -87.6267288762959,
              41.89242880348867
            ],
            [
              -87.6278035928968,
              41.89280724210791
            ],
            [
              -87.62892297568122,
              41.893105432465575
            ],
            [
              -87.630076240649,
              41.89332050168519
            ],
            [
              -87.63125227696989,
              41.8934503776852
            ],
            [
              -87.6324397541506,
              41.893493809166905
            ]
          ]
        ]
      }
    }

honolulugeo = {
"geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -157.86034709354786,
              21.315967329456527
            ],
            [
              -157.86129336277003,
              21.31592401976519
            ],
            [
              -157.86223051724133,
              21.315794507863405
            ],
            [
              -157.86314953005484,
              21.315580041247294
            ],
            [
              -157.86404154914294,
              21.315282685714163
            ],
            [
              -157.8648979825842,
              21.31490530545359
            ],
            [
              -157.8657105813979,
              21.31445153544481
            ],
            [
              -157.86647151902687,
              21.313925746426733
            ],
            [
              -157.86717346674172,
              21.31333300277897
            ],
            [
              -157.86780966423913,
              21.3126790137205
            ],
            [
              -157.86837398475356,
              21.311970078297037
            ],
            [
              -157.86886099405484,
              21.311213024688012
            ],
            [
              -157.8692660027644,
              21.310415144418908
            ],
            [
              -157.86958511148595,
              21.309584122113538
            ],
            [
              -157.86981524831828,
              21.308727961463823
            ],
            [
              -157.86995419838968,
              21.30785490813097
            ],
            [
              -157.87000062513178,
              21.306973370321142
            ],
            [
              -157.86995408309,
              21.306091837801215
            ],
            [
              -157.86981502214977,
              21.305218800134792
            ],
            [
              -157.8695847831402,
              21.30436266492598
            ],
            [
              -157.8692655848595,
              21.30353167685829
            ],
            [
              -157.8688605026507,
              21.302733838307937
            ],
            [
              -157.86837343873447,
              21.301976832295484
            ],
            [
              -157.8678090845884,
              21.30126794851731
            ],
            [
              -157.86717287573492,
              21.300614013168158
            ],
            [
              -157.86647093937611,
              21.300021323229704
            ],
            [
              -157.86571003537884,
              21.299495585856924
            ],
            [
              -157.86489749118005,
              21.29904186344471
            ],
            [
              -157.86404113123803,
              21.298664524902893
            ],
            [
              -157.86314920170906,
              21.29836720360744
            ],
            [
              -157.86223029107282,
              21.298152762432235
            ],
            [
              -157.86129324747031,
              21.29802326619688
            ],
            [
              -157.86034709354786,
              21.29797996179545
            ],
            [
              -157.8594009396254,
              21.29802326619688
            ],
            [
              -157.85846389602287,
              21.298152762432235
            ],
            [
              -157.85754498538662,
              21.29836720360744
            ],
            [
              -157.85665305585766,
              21.298664524902893
            ],
            [
              -157.85579669591567,
              21.29904186344471
            ],
            [
              -157.85498415171685,
              21.299495585856924
            ],
            [
              -157.85422324771957,
              21.300021323229704
            ],
            [
              -157.85352131136077,
              21.300614013168158
            ],
            [
              -157.85288510250732,
              21.30126794851731
            ],
            [
              -157.85232074836122,
              21.301976832295484
            ],
            [
              -157.851833684445,
              21.302733838307937
            ],
            [
              -157.8514286022362,
              21.30353167685829
            ],
            [
              -157.85110940395552,
              21.30436266492598
            ],
            [
              -157.85087916494592,
              21.305218800134792
            ],
            [
              -157.85074010400572,
              21.306091837801215
            ],
            [
              -157.85069356196394,
              21.306973370321142
            ],
            [
              -157.850739988706,
              21.30785490813097
            ],
            [
              -157.8508789387774,
              21.308727961463823
            ],
            [
              -157.85110907560977,
              21.309584122113538
            ],
            [
              -157.85142818433133,
              21.310415144418908
            ],
            [
              -157.85183319304085,
              21.311213024688012
            ],
            [
              -157.85232020234216,
              21.311970078297037
            ],
            [
              -157.85288452285656,
              21.3126790137205
            ],
            [
              -157.85352072035397,
              21.31333300277897
            ],
            [
              -157.85422266806881,
              21.313925746426733
            ],
            [
              -157.8549836056978,
              21.31445153544481
            ],
            [
              -157.8557962045115,
              21.31490530545359
            ],
            [
              -157.85665263795275,
              21.315282685714163
            ],
            [
              -157.85754465704085,
              21.315580041247294
            ],
            [
              -157.85846366985436,
              21.315794507863405
            ],
            [
              -157.8594008243257,
              21.31592401976519
            ],
            [
              -157.86034709354786,
              21.315967329456527
            ]
          ]
        ]
      }
    }

In [395]:
# gamingdataframe.to_csv("data/gaming.csv")
# gamingcsv = pd.read_csv("data/gaming.csv", index_col=0)

# schoolsdataframe.to_csv("data/schools.csv")
# schoolscsv = pd.read_csv("data/schools.csv", index_col=0)

# startupdataframe.to_csv("data/startup.csv")
# startupcsv = pd.read_csv("data/startup.csv", index_col=0)

# starbucksdataframe.to_csv("data/starbucks.csv")
# starbuckscsv = pd.read_csv("data/starbucks.csv", index_col=0)

# travellingdataframe.to_csv("data/travelling.csv")
# travellingcsv = pd.read_csv("data/travelling.csv", index_col=0)

# clubsdataframe.to_csv("data/clubs.csv")
# clubscsv = pd.read_csv("data/clubs.csv", index_col=0)

# vegandataframe.to_csv("data/vegan.csv")
# vegancsv = pd.read_csv("data/vegan.csv", index_col=0)

# basketballdataframe.to_csv("data/basketball.csv")
# basketballcsv = pd.read_csv("data/basketball.csv", index_col=0)

# dogcaredataframe.to_csv("data/dogcare.csv")
# dogcarecsv = pd.read_csv("data/dogcare.csv", index_col=0)

# gamingconcat = pd.concat([gamingcsv, gamingdataframe], ignore_index=True)
# gamingconcat.to_csv("data/gaming.csv")

# schoolsconcat = pd.concat([schoolscsv, schoolsdataframe], ignore_index=True)
# schoolsconcat.to_csv("data/schools.csv")

# startupconcat = pd.concat([startupcsv, startupdataframe], ignore_index=True)
# startupconcat.to_csv("data/startup.csv")

# starbucksconcat = pd.concat([starbuckscsv, starbucksdataframe], ignore_index=True)
# starbucksconcat.to_csv("data/starbucks.csv")

# travellingconcat = pd.concat([travellingcsv, travellingdataframe], ignore_index=True)
# travellingconcat.to_csv("data/trvelling.csv")

# clubsconcat = pd.concat([clubscsv, clubsdataframe], ignore_index=True)
# clubsconcat.to_csv("data/clubs.csv")

# veganconcat = pd.concat([vegancsv, vegandataframe], ignore_index=True)
# veganconcat.to_csv("data/vegan.csv")

# basketballconcat = pd.concat([basketballcsv, basketballdataframe], ignore_index=True)
# basketballconcat.to_csv("data/basketball.csv")

# dogcareconcat = pd.concat([dogcarecsv, dogcaredataframe], ignore_index=True)
# dogcareconcat.to_csv("data/dogcare.csv")

In [416]:
client2 = MongoClient("localhost:27017")
db2 = client.get_database("project3")

gamingmongo = db2.get_collection("gaming")
basketballmongo = db2.get_collection("basketball")
clubsmongo = db2.get_collection("clubs")
dogcaremongo = db2.get_collection("dogcare")
schoolsmongo = db2.get_collection("schools")
starbucksmongo = db2.get_collection("starbucks")
startupmongo = db2.get_collection("startup")
travellingmongo = db2.get_collection("travelling")
veganmongo = db2.get_collection("vegan")

sfgeo = list(db2.get_collection("sfgeo").find())[0]["geometry"]
chgeo = list(db2.get_collection("chgeo").find())[0]["geometry"]
hogeo = list(db2.get_collection("hogeo").find())[0]["geometry"]

In [478]:
len(list(starbucksmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": chgeo
                    }
                }
})))

30

In [489]:
## 88 employees
# (186 total relevance)
# 10 < (10)
# 8 < (9)
# 6 < (8)
# 4 < (7)
# 1 < (5)
# 0 < (0)

In [557]:
geo = hogeo

In [558]:
# 1. Gaming: 20

x = len(list(gamingmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
}))) #5

if x > 8:
    g = 10
elif x > 6:
    g = 9
elif x > 4:
    g = 8
elif x > 2:
    g = 7
elif x > 1:
    g = 5
else:
    g = 0

In [559]:
# 2. Schools: 26

x = len(list(schoolsmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
})))

if x > 8:
    sch = 10
elif x > 6:
    sch = 9
elif x > 4:
    sch = 8
elif x > 2:
    sch = 7
elif x > 1:
    sch = 5
else:
    sch = 0

In [560]:
# 3. Startup: 20

x = len(list(startupmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
}))) 

if x > 8:
    stup = 10
elif x > 6:
    stup = 9
elif x > 4:
    stup = 8
elif x > 2:
    stup = 7
elif x > 1:
    stup = 5
else:
    stup = 0

In [561]:
# 4. Starbucks: 10

x = len(list(starbucksmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
})))

if x > 8:
    sbck = 10
elif x > 6:
    sbck = 9
elif x > 4:
    sbck = 8
elif x > 2:
    sbck = 7
elif x > 1:
    sbck = 5
else:
    sbck = 0

In [562]:
# 5. Travelling: 20

x = len(list(travellingmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
})))

if x > 8:
    t = 10
elif x > 6:
    t = 9
elif x > 4:
    t = 8
elif x > 2:
    t = 7
elif x > 1:
    t = 5
else:
    t = 0

In [563]:
# 6. Clubs: 87

x = len(list(clubsmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
})))

if x > 8:
    c = 10
elif x > 6:
    c = 9
elif x > 4:
    c = 8
elif x > 2:
    c = 7
elif x > 1:
    c = 5
else:
    c = 0

In [564]:
# 7. Vegan: 1

x = len(list(veganmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
})))

if x > 8:
    v = 10
elif x > 6:
    v = 9
elif x > 4:
    v = 8
elif x > 2:
    v = 7
elif x > 1:
    v = 5
else:
    v = 0

In [565]:
# 8. Basketball: 1

x = len(list(basketballmongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
})))

if x > 8:
    b = 10
elif x > 6:
    b = 9
elif x > 4:
    b = 8
elif x > 2:
    b = 7
elif x > 1:
    b = 5
else:
    b = 0

In [566]:
# 9. Dog Care: 1

x = len(list(dogcaremongo.find({
                "location": {
                    "$geoWithin": {
                        "$geometry": geo
                    }
                }
})))

if x > 8:
    d = 10
elif x > 6:
    d = 9
elif x > 4:
    d = 8
elif x > 2:
    d = 7
elif x > 1:
    d = 5
else:
    d = 0

In [567]:
((g * 0.1 * 20) + (sch * 0.1 * 26) + (stup * 0.1 * 20) + (sbck * 0.1 * 10) + (t * 0.1 * 20) + (c * 0.1 * 87) + (v * 0.1 * 1) + (b * 0.1 * 1) + (d * 0.1 * 1))/186

0.4940860215053764